In [55]:
import pandas as pd   # Importa la biblioteca pandas
import numpy as np    # Importa la biblioteca numpy
import locale # Importa la biblioteca locale para la fecha

# Utilizamos el set_option para ver todas las columnas del fichero
pd.set_option('display.max_columns', None)

# Configura las fechas para español
try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
except locale.Error:
    print("Locale 'es_ES.UTF-8' no disponible.")

# Leer el archivo 'campaign_desc.csv' y crear el DataFrame asociado
df_campaign_desc= pd.read_csv('../data/campaign_desc.csv', sep=',')

# Leer el archivo 'campaign_table.csv' y crear el DataFrame asociado
df_campaign_table= pd.read_csv('../data/campaign_table.csv', sep=',')

# Leer el archivo 'coupon.csv' y crear el DataFrame asociado
df_coupon = pd.read_csv('../data/coupon.csv', sep=',')

# Leer el archivo 'coupon_redempt.csv' y crear el DataFrame asociado
df_coupon_redempt = pd.read_csv('../data/coupon_redempt.csv', sep=',')

# Leer el archivo 'hh_demographic.csv' y crear el DataFrame asociado
df_demographics = pd.read_csv('../data/hh_demographic.csv', sep=',')

# Leer el archivo 'product.csv' y crear el DataFrame asociado
df_product = pd.read_csv('../data/product.csv', sep=',')

# Leer el archivo 'transaction_data.csv' y crear el DataFrame asociado
df_transactions = pd.read_csv('../data/transaction_data.csv', sep=',')

# Leer el archivo 'casual_data.csv' y crear el DataFrame asociado
df_causal = pd.read_csv('../data/causal_data.csv', sep=',')

In [56]:
def limpieza_ficheros(df, contexto):
    '''
    Esta función elimina filas duplicadas de un DataFrame.
    Modifica campos que están a nulos por 'Unknown', cambios de formato
    Creación de campos nuevos, borrado de campos que no vamos a necesitar, renombrado de campos, filtrado del fichero
    
    Parámetros:
    - df: DataFrame que se desea limpiar.
    - contexto: El nombre que se dará al df para que renombre de forma correcta el campo del fichero
 
    Devuelve:
    - df: DataFrame limpio.
    '''

    # 1. Eliminacion de duplicados  
    df = df.drop_duplicates()

    # 2. Sustitución de ' ' por 'Unknown' 
    for col in df.select_dtypes(include='object').columns:
        # Convertir a string y eliminar espacios para identificar celdas vacías o solo espacios
        mask_espacios = df[col].astype(str).str.strip() == ''
        # Reemplazar esas celdas por 'Unknown'
        df.loc[mask_espacios, col] = 'Unknown'
    
    # 3. Convierte los nombres de columnas a minúsculas y reemplaza espacios por guiones bajos (_).
    df.columns = (df.columns
        .str.strip()                 # Quita espacios al inicio y final
        .str.lower()                # Convierte a minúsculas
        .str.replace(' ', '_')      # Reemplaza espacios por guiones bajos
        .str.replace(r'[^\w_]', '', regex=True)  # Elimina caracteres especiales
    )

    # 4. Borramos columnas que no vamos a utilizar
    if contexto == 'product':
        df = df.drop('curr_size_of_product', axis=1)
    if contexto == 'demographics':
        df = df.drop('kid_category_desc', axis=1)
    if contexto == 'coupon_redempt':
        df = df.drop('day', axis=1)
    if contexto == 'transactions':
        df = df.drop('trans_time', axis=1)

    # 5. Filtramos el fichero para recuperar los productos que se han vendido
    if contexto == 'transactions':
        df = df[(df['sales_value'] > 0) & (df['quantity'] > 0)]

    # 6. Renombrado de columnas
    if 'campaign' in df.columns:
        df = df.rename(columns={'campaign': 'campaign_id'})
    if 'description' in df.columns:
        df = df.rename(columns={'description': 'campaign_type'})
    if 'household_key' in df.columns:
        df = df.rename(columns={'household_key': 'household_id'})
    if 'day' in df.columns:
        df = df.rename(columns={'day': 'transaction_day'})
    if contexto == 'campaign_desc':
        df = df.rename(columns={'start_day': 'campaign_start_date'})
        df = df.rename(columns={'end_day': 'campaign_end_date'})

    # 7. Cálculo de fechas en formato 'YYYY-MM-DD'

    # Asume que el día 0 es el 1 de enero de 2017
    base_date = pd.to_datetime('2017-01-01')

    # Modifican las columnas a con fechas reales
    if contexto == 'campaign_desc':
        df['campaign_start_date'] = base_date + pd.to_timedelta(df['campaign_start_date'], unit='D')
        df['campaign_end_date'] = base_date + pd.to_timedelta(df['campaign_end_date'], unit='D')
    if contexto == 'transactions':
        df['transaction_day'] = base_date + pd.to_timedelta(df['transaction_day'], unit='D')
    
    # 8. Cambiamos el formato de los siguientes campos a positivo
    if contexto == 'transactions':
        df['retail_disc'] = df['retail_disc'].abs()
        df['coupon_match_disc'] = df['coupon_match_disc'].abs()

    # 9. Creamos los siguientes campos
    if contexto == 'transactions':
        # Descuento total ajustado para no superar sales_value
        effective_discount = np.minimum(df['retail_disc'] + df['coupon_match_disc'], df['sales_value'])
        # Precio por unidad (real pagado por cliente)
        df['actual_unit_price'] = round(((df['sales_value'] - effective_discount)/ df['quantity']),2)
        # Precio original de estantería (sin descuentos)
        df['shelf_unit_price'] = round(((df['sales_value'] + effective_discount)/ df['quantity']),2)

        # Crea una columna booleana o binaria: 1 si hubo algún descuento por cupón, 0 si no
        df["used_coupon"] = ((df["coupon_disc"] > 0) | (df["coupon_match_disc"] > 0)).astype(int)

    # se añade el campo de descripcion de display y descripcion de mailer
    descripcion_display = {
        '0': 'Not on display',
        '1': 'Store front',
        '2': 'Store rear',
        '3': 'Front end cap',
        '4': 'Mid-aisle end cap',
        '5': 'Rear end cap',
        '6': 'Side-aisle end cap',
        '7': 'In-aisle',
        '9': 'Secondary location display',
        'A': 'In-shelf'
    }
    descripcion_mailer= {
        '0': 'Not on ad',
        'A': 'Interior page feature',
        'C': 'Interior page line item',
        'D': 'Front page feature',
        'F': 'Back page feature',
        'H': 'Wrap front feature',
        'J': 'Wrap interior coupon',
        'L': 'Wrap back feature',
        'P': 'Interior page coupon',
        'X': 'Free on interior page',
        'Z': 'Free on front page, back page or wrap'
    }
    if contexto == 'causal':
        df['display_desc'] = df['display'].map(descripcion_display)
        df['mailer_desc'] = df['mailer'].map(descripcion_mailer)
    
    # Creamos el campo season en el fichero df_campaign_desc
    if contexto == 'campaign_desc':
        df['season'] = df['campaign_start_date'].apply(
        lambda fecha: (
            f'Winter-{str(fecha.year)[-2:]}' if (fecha.month == 12 and fecha.day >= 21) or (1 <= fecha.month <= 2) or (fecha.month == 3 and fecha.day < 21)
            else f'Spring-{str(fecha.year)[-2:]}' if (fecha.month == 3 and fecha.day >= 21) or (4 <= fecha.month <= 5) or (fecha.month == 6 and fecha.day < 21)
            else f'Summer-{str(fecha.year)[-2:]}' if (fecha.month == 6 and fecha.day >= 21) or (7 <= fecha.month <= 8) or (fecha.month == 9 and fecha.day < 21)
            else f'Autumn-{str(fecha.year)[-2:]}'
        )
    )
        
    return df

In [57]:
# Aplica la función a cada uno de tus DataFrames
df_campaign_desc = limpieza_ficheros(df_campaign_desc, 'campaign_desc')
df_campaign_table = limpieza_ficheros(df_campaign_table, 'campaign_table')
df_coupon = limpieza_ficheros(df_coupon, 'coupon')
df_coupon_redempt = limpieza_ficheros(df_coupon_redempt, 'coupon_redempt')
df_demographics = limpieza_ficheros(df_demographics, 'demographics')
df_product = limpieza_ficheros(df_product, 'product')
df_transactions = limpieza_ficheros(df_transactions,'transactions')
df_causal = limpieza_ficheros(df_causal, 'causal')

In [58]:
# Vamos realizando merges para juntar los dfs
mg_campaign = pd.merge(df_campaign_table, df_campaign_desc, on=('campaign_id','campaign_type'), how='inner')
mg_coupon = pd.merge(df_coupon_redempt, df_coupon, on=('campaign_id', 'coupon_upc'), how='inner')
mg_camp_demo = pd.merge(mg_campaign, df_demographics, on='household_id', how='inner')
mg_camp_demo_cou = pd.merge(mg_camp_demo, mg_coupon, on=('household_id','campaign_id'), how='inner')

# Dejamos entre otros el siguiente fichero final
df_household = pd.merge(df_transactions, mg_camp_demo_cou, on=('household_id', 'product_id'), how='inner')

# **Export ficheros limpios**

In [59]:
# Exportamos el fichero después de su limpieza y creación de variables a csv
df_household.to_csv('../results/archivo_limpio_household.csv',sep=';', decimal=',', index=False) 
# Exportamos los siguientes dos ficheros después de su limpieza y creación de variables a csv sin realizar merge
df_product.to_csv('../results/archivo_limpio_product.csv', index=False)
df_causal.to_csv('../results/archivo_limpio_causal.csv', index=False) 